In [6]:
import re
import pandas as pd
from pandas import concat
import openpyxl
import openpyxl.styles



def parse_pin_file(file_path):
    # Define column names
    columns = ['PinNumber', 'PinName', 'Mode', 'Signal']

    # Create an empty DataFrame with the specified columns
    pin_data = pd.DataFrame(columns=columns)
    current_pin_number = None
    current_pin_name = None

    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        if line.startswith("Mcu.Pin"):
            pin_match = re.match(r'Mcu\.Pin(\d+)=(\w+)', line)
            if pin_match:
                current_pin_number, current_pin_name = pin_match.groups() if pin_match else (None, None)
                current_pin_number = int(current_pin_number)
                # Add a new row
                new_data = {'PinNumber': current_pin_number, 'PinName': current_pin_name, 'Mode': '', 'Signal': ''}
                pin_data = pd.concat([pin_data, pd.DataFrame([new_data])], ignore_index=True)
                #pin_data[current_pin_number] = {'PinName': current_pin_name}
        elif "RCC_OSC_IN" in line:
            new_data = {'PinNumber': '', 'PinName': line[:3], 'Mode': '', 'Signal': "RCC_OSC_IN"}
            pin_data = pd.concat([pin_data, pd.DataFrame([new_data])], ignore_index=True)
        elif "RCC_OSC_OUT" in line:
            new_data = {'PinNumber': '', 'PinName': line[:3], 'Mode': '', 'Signal': "RCC_OSC_OUT"}
            pin_data = pd.concat([pin_data, pd.DataFrame([new_data])], ignore_index=True)
    for line in lines:
        #if current_pin_name is not None and line.startswith(current_pin_name):
        for pin_name in pin_data['PinName']:
            mode_match = re.match(r'{}\.(Mode)=(\w+)'.format(pin_name), line)
            signal_match = re.match(r'{}\.(Signal)=(\w+)'.format(pin_name), line)
            if not mode_match and not signal_match:
                continue
            match = mode_match if mode_match else signal_match
            i = pin_data[pin_data['PinName'] == pin_name].index[0]
            pin_data.loc[i, 'Mode' if mode_match else 'Signal'] = match.group(2)

    return pin_data

def write_excel_file(excel_file_path, pin_data):
    # Export the DataFrame to Excel with alternating row colors, borders, and adjusted column widths
    with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
        pin_data.to_excel(writer, sheet_name='Sheet1', index=False)

        # Access the Excel writer and the sheet
        workbook = writer.book
        sheet = writer.sheets['Sheet1']

        # Define the colors
        color_header = '4285F4'  # Dark blue
        color_white = 'FFFFFF'  # White
        color_light_blue = 'DDEBF7'  # Light blue

        # Apply color for the header (first row)
        for col_idx in range(1, sheet.max_column + 1):
            sheet.cell(row=1, column=col_idx).fill = openpyxl.styles.PatternFill(
                start_color=color_header, end_color=color_header, fill_type='solid'
            )

        # Apply alternating row colors starting from the second row
        for row_idx in range(2, sheet.max_row + 1, 2):
            for col_idx in range(1, sheet.max_column + 1):
                sheet.cell(row=row_idx, column=col_idx).fill = openpyxl.styles.PatternFill(
                    start_color=color_white, end_color=color_white, fill_type='solid'
                )

        for row_idx in range(3, sheet.max_row + 1, 2):
            for col_idx in range(1, sheet.max_column + 1):
                sheet.cell(row=row_idx, column=col_idx).fill = openpyxl.styles.PatternFill(
                    start_color=color_light_blue, end_color=color_light_blue, fill_type='solid'
                )

        # Adjust column widths to fit the data
        for col in sheet.columns:
            max_length = 0
            column = col[0].column_letter  # Get the column name
            for cell in col:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2)
            sheet.column_dimensions[column].width = adjusted_width

        # Add thin lines separating rows and columns
        thin_border = openpyxl.styles.Side(border_style='thin', color='000000')
        border = openpyxl.styles.Border(left=thin_border, right=thin_border, top=thin_border, bottom=thin_border)

        for row in sheet.iter_rows():
            for cell in row:
                cell.border = border
        print(f'DataFrame exported to {excel_file_path} with alternating row colors.')


if __name__ == "__main__":
    file_path = "STM32F722IET7_Pinout.ioc"  # Replace with the actual path to your file

    pin_data = parse_pin_file(file_path)


    # Display the resulting DataFrame
    #print(pin_data)

    # Specify the Excel file path
    excel_file_path = file_path[:-3]+"xlsx"

    write_excel_file(excel_file_path, pin_data)


DataFrame exported to STM32F722IET7_Pinout.xlsx with alternating row colors.
